In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gmaps

# Google developer API key
from config import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

#import County Level Coordinate Data with ANSI encoding
csv_path_county = "../data/County_Coordinates.csv"
county_df = pd.read_csv(csv_path_county, delimiter=",",encoding="ANSI")
county_df.head()

,county,state,population,lat,lon,lat_gmap,lon_gmap
0,Abbeville County,sc,16713,34.301500,-82.426750,34.189122,-82.475276
1,Acadia Parish,la,44719,30.249556,-92.393000,30.229735,-92.381362
2,Accomack County,va,33115,37.784250,-75.649750,37.706332,-75.806908
3,Ada County,id,417501,43.607966,-116.273847,43.478767,-116.242030
4,Adair County,ia,4561,41.280000,-94.485500,41.274173,-94.479976


In [2]:
#import cleaned data at City Level with ANSI encoding

csv_path_clean = "../data/Cleaned_Data.csv"
solar_clean_df = pd.read_csv(csv_path_clean, delimiter=",", encoding="ANSI")
solar_clean_df.head()

,Unnamed: 0,tile_count,solar_system_count,total_panel_area,fips,average_household_income,county,education_bachelor,education_college,education_doctoral,...,incentive_residential_state_level,incentive_nonresidential_state_level,net_metering,feedin_tariff,cooperate_tax,property_tax,sales_tax,rebate,avg_electricity_retail_rate,Solar_Status
0,0,0,0,0.000000,27145011200,70352.78987,Stearns County,569,1690,13,...,11,13,34,0,0,25,12,0,9.46,No
1,2,3,3,64.505776,27145011302,71496.88658,Stearns County,854,1459,31,...,11,13,34,0,0,25,12,0,9.46,Yes
2,3,0,0,0.000000,27145011304,86840.15275,Stearns County,640,1116,68,...,11,13,34,0,0,25,12,0,9.46,No
3,4,5,5,164.583303,27145011400,89135.31560,Stearns County,654,1314,15,...,11,13,34,0,0,25,12,0,9.46,Yes
4,5,0,0,0.000000,27145011500,62225.90361,Stearns County,522,1395,24,...,11,13,34,0,0,25,12,0,9.46,No


In [3]:
#create new column for total county income
solar_clean_df["county_household_income"]=solar_clean_df["population"]*solar_clean_df["average_household_income"]
solar_clean_df.head()

,Unnamed: 0,tile_count,solar_system_count,total_panel_area,fips,average_household_income,county,education_bachelor,education_college,education_doctoral,...,incentive_nonresidential_state_level,net_metering,feedin_tariff,cooperate_tax,property_tax,sales_tax,rebate,avg_electricity_retail_rate,Solar_Status,county_household_income
0,0,0,0,0.000000,27145011200,70352.78987,Stearns County,569,1690,13,...,13,34,0,0,25,12,0,9.46,No,4.627103e+08
1,2,3,3,64.505776,27145011302,71496.88658,Stearns County,854,1459,31,...,13,34,0,0,25,12,0,9.46,Yes,4.984048e+08
2,3,0,0,0.000000,27145011304,86840.15275,Stearns County,640,1116,68,...,13,34,0,0,25,12,0,9.46,No,5.495245e+08
3,4,5,5,164.583303,27145011400,89135.31560,Stearns County,654,1314,15,...,13,34,0,0,25,12,0,9.46,Yes,4.692083e+08
4,5,0,0,0.000000,27145011500,62225.90361,Stearns County,522,1395,24,...,13,34,0,0,25,12,0,9.46,No,4.063974e+08


In [4]:
#new groupby object 

#grouby by county and state

grouped_location = solar_clean_df.groupby(["county", "state"])

#create empty dataframe for sorted county and state data

location_df = pd.DataFrame()

#pull in target variable data
location_df["tile_count_per1000"]=grouped_location["tile_count"].sum()*1000/grouped_location["population"].sum()
location_df["solar_system_count_per1000"]=grouped_location["solar_system_count"].sum()*1000/grouped_location["population"].sum()
location_df["total_panel_area_per1000"]=grouped_location["total_panel_area"].sum()*1000/grouped_location["population"].sum()
location_df["total_panel_area"]=grouped_location["total_panel_area"].sum()

#pull in independent variables on welfare
location_df["average_household_income"]=grouped_location["county_household_income"].sum()/grouped_location["population"].sum()
location_df["per_capita_income"]=grouped_location["per_capita_income"].mean()
location_df["median_household_income"]=grouped_location["median_household_income"].mean()
location_df["housing_unit_median_value"]=grouped_location["housing_unit_median_value"].mean()

#pull in independent variables on weather conditions
location_df["daily_solar_radiation"]=grouped_location["daily_solar_radiation"].mean()


#reset index on location_df to ensure county names appear in each row
location_df=location_df.reset_index()
location_df.head()


,county,state,tile_count_per1000,solar_system_count_per1000,total_panel_area_per1000,total_panel_area,average_household_income,per_capita_income,median_household_income,housing_unit_median_value,daily_solar_radiation
0,Abbeville County,sc,0.179501,0.179501,3.303239,55.207033,49709.200865,19624.750000,37707.250000,101050.000000,4.31
1,Acadia Parish,la,1.252264,1.028646,51.718155,2312.784175,54859.088846,20804.666667,40660.888889,90300.000000,4.36
2,Accomack County,va,0.241582,0.241582,8.194007,271.344537,54158.288332,23293.375000,39467.000000,151862.500000,4.37
3,Ada County,id,1.607182,1.293410,46.063401,19231.515912,76074.034496,29962.338983,59137.406780,199111.864407,4.45
4,Adair County,ia,1.096251,1.096251,124.555987,568.099858,58670.624386,27300.000000,47743.000000,92750.000000,3.94


In [5]:
#merge dataframes by county and state together
new_df=pd.merge(county_df, location_df, how="left", on=["county", "state"])
new_df.head()

,county,state,population,lat,lon,lat_gmap,lon_gmap,tile_count_per1000,solar_system_count_per1000,total_panel_area_per1000,total_panel_area,average_household_income,per_capita_income,median_household_income,housing_unit_median_value,daily_solar_radiation
0,Abbeville County,sc,16713,34.301500,-82.426750,34.189122,-82.475276,0.179501,0.179501,3.303239,55.207033,49709.200865,19624.750000,37707.250000,101050.000000,4.31
1,Acadia Parish,la,44719,30.249556,-92.393000,30.229735,-92.381362,1.252264,1.028646,51.718155,2312.784175,54859.088846,20804.666667,40660.888889,90300.000000,4.36
2,Accomack County,va,33115,37.784250,-75.649750,37.706332,-75.806908,0.241582,0.241582,8.194007,271.344537,54158.288332,23293.375000,39467.000000,151862.500000,4.37
3,Ada County,id,417501,43.607966,-116.273847,43.478767,-116.242030,1.607182,1.293410,46.063401,19231.515912,76074.034496,29962.338983,59137.406780,199111.864407,4.45
4,Adair County,ia,4561,41.280000,-94.485500,41.274173,-94.479976,1.096251,1.096251,124.555987,568.099858,58670.624386,27300.000000,47743.000000,92750.000000,3.94


In [6]:
#take random 1000 data points for graphing purposes enables a smaller file size
randomize=new_df.sample(n=1000)
#reset index so it's in numerical order
random_df=randomize.reset_index()

random_df.head()


,index,county,state,population,lat,lon,lat_gmap,lon_gmap,tile_count_per1000,solar_system_count_per1000,total_panel_area_per1000,total_panel_area,average_household_income,per_capita_income,median_household_income,housing_unit_median_value,daily_solar_radiation
0,518,Clark County,ky,28981,37.993125,-84.166500,37.959945,-84.143514,1.035161,0.966150,45.462216,1317.540491,58145.314923,23951.500000,44111.000000,135562.500000,3.905000
1,2353,Rawlins County,ks,2545,39.785000,-101.076000,39.860090,-101.071176,0.000000,0.000000,0.000000,0.000000,57563.366340,27559.000000,41579.000000,75600.000000,4.540000
2,2725,Tehama County,ca,63152,40.106727,-122.211636,40.098220,-122.174605,11.448569,7.252344,485.657349,30670.232876,54993.174460,21673.818182,42750.000000,176409.090909,4.844545
3,2924,Washington County,ri,96473,41.452182,-71.589864,41.456811,-71.667335,5.203528,2.187140,376.218854,36294.961506,92593.530146,40277.318182,73760.727273,374622.727273,3.810000
4,594,Collingsworth County,tx,3058,34.965000,-100.271000,35.035162,-100.349789,2.616089,1.962067,237.499473,726.273388,51131.559960,20269.000000,39743.000000,64200.000000,4.900000


In [7]:
#create_map function for two layered map
#x is heat map variable
#y is symbol map variable

def create_map(x,y):
    
    # store the figure in a variable so we can continue to add layers to the same map
    fig = gmaps.figure()
    
    #coordinates on gmap lat and lon for each county
    coordinates=random_df[['lat_gmap', 'lon_gmap']]
    
    #density is proportial to column values associated with x in function
    density = random_df[x].astype(float)

    # Create a marker layer using our coordinates
    heat_layer = gmaps.heatmap_layer(coordinates, weights=density, dissipating=True)

    # Add the layer to the map
    fig.add_layer(heat_layer)
    
    #pull in column data for marker map
    data = random_df[y]
    
    # Normalize the data from column["y"] from 0 to 1 and store that in a new variable
    norm_data = (data - min(data)) / (max(data) - min(data))
    
    #plot histogram of normalized data for variable y
    #norm_data.hist()
    
    #create data layer for variable y
    data_layer = gmaps.symbol_layer(coordinates, 
        fill_color="green",
        stroke_color='navy',
        fill_opacity=norm_data,
        stroke_opacity=norm_data,
        info_box_content=[f"y: {y}" for y in data])
                                   

    # Add the layer to the figure
    fig.add_layer(data_layer)

    # Display the figure
    return fig

In [8]:
create_map("housing_unit_median_value","total_panel_area_per1000")

Figure(layout=FigureLayout(height='420px'))

In [9]:
#save dataframe chosen values to csv
new_df.to_csv("../data/Solar_Data.csv", index=False, header=True)